In [3]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report
import numpy as np
import os
import pandas as pd
from scipy import stats
import matplotlib.pyplot as plt
import plotly.graph_objects as go
import tarfile
import zipfile

In [6]:
# Carga los datos de entrenamiento y prueba
train_data = pd.read_csv('./data/train.csv')
test_data = pd.read_csv('./data/test.csv')

# Preprocesamiento y TF-IDF para entrenamiento
tfidf_vectorizer = TfidfVectorizer(
    max_features=1000,  # Limita el número de características
    stop_words='english',  # Elimina stopwords en inglés
    ngram_range=(1, 2),  # Utiliza bigramas además de unigramas
    sublinear_tf=True,  # Aplica sublinear TF scaling
)
X_train_tfidf = tfidf_vectorizer.fit_transform(train_data['titulo'])
y_train = train_data['categoria']

# División de datos de entrenamiento y validación
X_train, X_valid, y_train, y_valid = train_test_split(X_train_tfidf, y_train, test_size=0.2, random_state=42)

# Entrenamiento del modelo de regresión logística
lr = LogisticRegression(
    C=1.0,  # Parámetro de regularización
    penalty='l2',  # Regularización L2
    solver='lbfgs',  # Solucionador
    max_iter=1000,  # Número máximo de iteraciones
)
lr.fit(X_train, y_train)

# Predicciones en datos de validación
y_pred = lr.predict(X_valid)

# Evaluación del modelo
print(classification_report(y_valid, y_pred))

# Preprocesamiento y TF-IDF para prueba
X_test_tfidf = tfidf_vectorizer.transform(test_data['titulo'])

# Predicciones en datos de prueba
y_pred_test = lr.predict(X_test_tfidf)

# Guarda los resultados en un DataFrame con la columna "ID"
results = pd.DataFrame({'ID': test_data['index'], 'categoria': y_pred_test})

# Guarda el DataFrame en un archivo CSV
results.to_csv('prediccionesRL.csv', index=False)

                      precision    recall  f1-score   support

            Deportes       0.70      0.92      0.80        38
           Educación       0.73      0.33      0.46        24
     Entretenimiento       0.69      0.79      0.74        48
       Gente y Blogs       0.50      0.12      0.20         8
               Otros       1.00      0.50      0.67        10
Película y Animación       0.54      0.62      0.58        32

            accuracy                           0.67       160
           macro avg       0.69      0.55      0.57       160
        weighted avg       0.68      0.67      0.65       160

